In [3]:
import pandas as pd

# Load the CSV file
file_path = '../data/raw/Bank_of_Canada_interest_rate_1980-2024.csv'
df = pd.read_csv(file_path)

# Melt the DataFrame to convert columns to rows
df_melted = df.melt(id_vars=['Financial market statistics'], var_name='Date', value_name='Rate')

# Replace '..' with NaN and then forward fill to replace with the previous value
df_melted['Rate'].replace('..', pd.NA, inplace=True)
df_melted['Rate'] = pd.to_numeric(df_melted['Rate'], errors='coerce')
df_melted['Rate'].fillna(method='ffill', inplace=True)

# Convert 'Date' column to datetime format
df_melted['Date'] = pd.to_datetime(df_melted['Date'], format='%d-%b-%y')

# Roll up days into month-year and calculate the average rate for each month
df_melted['Month-Year'] = df_melted['Date'].dt.to_period('M')
monthly_avg_df = df_melted.groupby(['Month-Year', 'Financial market statistics'])['Rate'].mean().reset_index()

# Pivot the data to have 'Month-Year' as index and 'Rate' as columns
df_pivoted = monthly_avg_df.pivot(index='Month-Year', columns='Financial market statistics', values='Rate')

# Reset index to flatten the DataFrame
df_pivoted.reset_index(inplace=True)

# Select only 'Month-Year' and 'Overnight money market financing' columns
final_df = df_pivoted[['Month-Year', 'Overnight money market financing']]

# Save the transformed dataset to a new CSV file
output_path = '../data/processed/Bank_of_Canada_interest_rate_198_.csv'
final_df.to_csv(output_path, index=False)

# Display the first few rows of the transformed dataset
print("Transformed DataFrame:")
print(final_df.head())


C:\Users\dimam\AppData\Local\Temp\ipykernel_7860\2480355804.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_melted['Rate'].replace('..', pd.NA, inplace=True)
C:\Users\dimam\AppData\Local\Temp\ipykernel_7860\2480355804.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Transformed DataFrame:
Financial market statistics Month-Year  Overnight money market financing
0                              1980-01                         13.270833
1                              1980-02                         11.982759
2                              1980-03                         11.427419
3                              1980-04                         15.841667
4                              1980-05                         17.052419


In [7]:
# add data to the database

import pandas as pd
import sqlite3

# Load the transformed CSV file
transformed_file_path = '../data/processed/Bank_of_Canada_interest_rate_1980_2024_.csv'
transformed_df = pd.read_csv(transformed_file_path)

# Connect to the SQLite database
db_path = '../data/sql/Canada_Housing_Prices_2005_2024.db'
conn = sqlite3.connect(db_path)

# Add the transformed data to the database
table_name = 'Bank_of_Canada_Interest_Rate'
transformed_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"Data has been added to the database as table '{table_name}'")


Data has been added to the database as table 'Bank_of_Canada_Interest_Rate'
